# Fusion 360 Gewindetool – XML-Editor mit Jupyter Notebook

Dieses Projekt hilft dir, benutzerdefinierte Gewinde für Autodesk Fusion 360 einfach in eine XML-Datei einzutragen.
Es verwendet ein interaktives Jupyter Notebook mit `ipywidgets`, um Eingaben auf Deutsch zu erfassen und korrekt in das englischsprachige XML-Format zu übertragen.

In [63]:
from IPython.display import display
import ipywidgets as widgets
from ipywidgets import Layout, HBox, VBox
import subprocess
import datetime
import threading
import time
from IPython.display import HTML
import xml.dom.minidom

# CSS hacks
display(HTML("<style>.highlighted select { border: 3px solid lightgreen !important; }</style>"))
display(widgets.HTML("<h1>Fusion 360 Gewindetool – XML-Editor mit Jupyter Notebook</h1>"))

# WICHTIG: feste XML-Datei im neuen data-Verzeichnis
xml_path = 'data/AstroISOmetric.xml'

def remove_blank_lines(xml_string):
    lines = xml_string.split('\n')
    non_empty_lines = [line for line in lines if line.strip() != '']
    return '\n'.join(non_empty_lines)

# XML-Formatierung
def pretty_print_xml(path):
    with open(path, 'r') as f:
        xml_string = f.read()
    parsed = xml.dom.minidom.parseString(xml_string)
    pretty_xml_as_string = parsed.toprettyxml(indent="  ")
    with open(path, 'w') as f:
        f.write(pretty_xml_as_string)

def load_dropdown_options():
    tree = ET.parse(xml_path)
    root = tree.getroot()
    options = ['<Neu laden>']
    for thread_size in root.findall('ThreadSize'):
        designation = thread_size.find('Designation/ThreadDesignation').text
        if designation:  # nur wenn nicht leer
            options.append(designation)
    thread_dropdown.options = options

def fill_fields_from_selection(change):
    selection = change['new']
    if selection == '<Neu laden>':
        size.value = 0
        designation_de.value = ''
        designation_en.value = ''
        pitch.value = 0
        major_dia_ext.value = 0
        pitch_dia_ext.value = 0
        minor_dia_ext.value = 0
        major_dia_int.value = 0
        pitch_dia_int.value = 0
        minor_dia_int.value = 0
        tap_drill.value = 0
        major_dia_4g6g.value = 0
        pitch_dia_4g6g.value = 0
        minor_dia_4g6g.value = 0
        return

    tree = ET.parse(xml_path)
    root = tree.getroot()
    for thread_size in root.findall('ThreadSize'):
        designation = thread_size.find('Designation/ThreadDesignation').text
        if designation == selection:
            size.value = float(thread_size.find('Size').text)
            designation_de.value = thread_size.find('Designation/CTD').text
            designation_en.value = designation
            pitch.value = float(thread_size.find('Designation/Pitch').text)

            threads = thread_size.find('Designation').findall('Thread')
            for t in threads:
                gender = t.find('Gender').text
                class_value = t.find('Class').text
                major = float(t.find('MajorDia').text)
                pitch_dia = float(t.find('PitchDia').text)
                minor = float(t.find('MinorDia').text)

                if gender == 'external' and class_value == '6g':
                    major_dia_ext.value = major
                    pitch_dia_ext.value = pitch_dia
                    minor_dia_ext.value = minor
                elif gender == 'internal' and class_value == '6H':
                    major_dia_int.value = major
                    pitch_dia_int.value = pitch_dia
                    minor_dia_int.value = minor
                    tap_drill.value = float(t.find('TapDrill').text)
                elif gender == 'external' and class_value == '4g6g':
                    major_dia_4g6g.value = major
                    pitch_dia_4g6g.value = pitch_dia
                    minor_dia_4g6g.value = minor
            break

def add_thread_to_xml(path, size, designation_de, designation_en, pitch,
                      major_dia_ext, pitch_dia_ext, minor_dia_ext,
                      major_dia_int, pitch_dia_int, minor_dia_int,
                      major_dia_4g6g, pitch_dia_4g6g, minor_dia_4g6g,
                      tap_drill):
    tree = ET.parse(path)
    root = tree.getroot()

    for ts in root.findall('ThreadSize'):
        desig = ts.find('Designation/ThreadDesignation').text
        if desig == designation_en:
            root.remove(ts)

    thread_size = ET.SubElement(root, 'ThreadSize')
    ET.SubElement(thread_size, 'Size').text = str(size)
    designation_el = ET.SubElement(thread_size, 'Designation')
    ET.SubElement(designation_el, 'ThreadDesignation').text = designation_en
    ET.SubElement(designation_el, 'CTD').text = designation_de
    ET.SubElement(designation_el, 'Pitch').text = str(pitch)

    thread_ext = ET.SubElement(designation_el, 'Thread')
    ET.SubElement(thread_ext, 'Gender').text = 'external'
    ET.SubElement(thread_ext, 'Class').text = '6g'
    ET.SubElement(thread_ext, 'MajorDia').text = str(major_dia_ext)
    ET.SubElement(thread_ext, 'PitchDia').text = str(pitch_dia_ext)
    ET.SubElement(thread_ext, 'MinorDia').text = str(minor_dia_ext)

    thread_int = ET.SubElement(designation_el, 'Thread')
    ET.SubElement(thread_int, 'Gender').text = 'internal'
    ET.SubElement(thread_int, 'Class').text = '6H'
    ET.SubElement(thread_int, 'MajorDia').text = str(major_dia_int)
    ET.SubElement(thread_int, 'PitchDia').text = str(pitch_dia_int)
    ET.SubElement(thread_int, 'MinorDia').text = str(minor_dia_int)
    ET.SubElement(thread_int, 'TapDrill').text = str(tap_drill)

    thread_ext_4g6g = ET.SubElement(designation_el, 'Thread')
    ET.SubElement(thread_ext_4g6g, 'Gender').text = 'external'
    ET.SubElement(thread_ext_4g6g, 'Class').text = '4g6g'
    ET.SubElement(thread_ext_4g6g, 'MajorDia').text = str(major_dia_4g6g)
    ET.SubElement(thread_ext_4g6g, 'PitchDia').text = str(pitch_dia_4g6g)
    ET.SubElement(thread_ext_4g6g, 'MinorDia').text = str(minor_dia_4g6g)

    tree.write(path)

    # Schön formatieren ohne Leerzeilen
    with open(path, 'r') as f:
        xml_str = f.read()
    pretty_xml = xml.dom.minidom.parseString(xml_str).toprettyxml(indent="  ")
    pretty_xml_clean = remove_blank_lines(pretty_xml)
    with open(path, 'w') as f:
        f.write(pretty_xml_clean)

    print(f"✅ Gewinde {designation_en} ({designation_de}) erfolgreich in {path} gespeichert und formatiert.")

def delete_thread_from_xml(designation_en):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    for ts in root.findall('ThreadSize'):
        desig = ts.find('Designation/ThreadDesignation').text
        if desig == designation_en:
            root.remove(ts)
            tree.write(xml_path)
            print(f"🗑️ Gewinde {designation_en} gelöscht.")
            return
    print(f"⚠️ Gewinde {designation_en} nicht gefunden.")


# Layouts (angepasst!)
field_layout = Layout(width='500px', min_width='500px', max_width='500px')
vbox_layout = Layout(width='550px', margin='0 40px 20px 0')  # noch mehr rechter Abstand
style = {'description_width': '350px'}  # mehr Platz für die Beschriftungen

# Widgets (verwenden field_layout + style)
thread_dropdown = widgets.Dropdown(description='Vorhandene Gewinde:', options=['<Neu laden>'], layout=field_layout, style=style)
size = widgets.FloatText(description='Nenndurchmesser (Size):', layout=field_layout, style=style)
designation_de = widgets.Text(description='Custom Thread designation - Freitext (CTD):', layout=field_layout, style=style)
designation_en = widgets.Text(description='Technische Bezeichnung (ThreadDesignation) z.B. M56x0.75:', layout=field_layout, style=style)
pitch = widgets.FloatText(description='Steigung (Pitch):', layout=field_layout, style=style)

# Layouts für die verschiedenen Gewinde (jetzt rechtsbündig)
label_ext_6g = widgets.HTML(value='<div style="text-align: right; width: 100%;"><b>External 6g</b></div>')
major_dia_ext = widgets.FloatText(description='MajorDia:', layout=field_layout, style=style)
pitch_dia_ext = widgets.FloatText(description='PitchDia:', layout=field_layout, style=style)
minor_dia_ext = widgets.FloatText(description='MinorDia:', layout=field_layout, style=style)

label_int_6H = widgets.HTML(value='<div style="text-align: right; width: 100%;"><b>Internal 6H</b></div>')
major_dia_int = widgets.FloatText(description='MajorDia:', layout=field_layout, style=style)
pitch_dia_int = widgets.FloatText(description='PitchDia:', layout=field_layout, style=style)
minor_dia_int = widgets.FloatText(description='MinorDia:', layout=field_layout, style=style)
tap_drill = widgets.FloatText(description='TapDrill:', layout=field_layout, style=style)

label_ext_4g6g = widgets.HTML(value='<div style="text-align: right; width: 100%;"><b>External 4g6g</b></div>')
major_dia_4g6g = widgets.FloatText(description='MajorDia:', layout=field_layout, style=style)
pitch_dia_4g6g = widgets.FloatText(description='PitchDia:', layout=field_layout, style=style)
minor_dia_4g6g = widgets.FloatText(description='MinorDia:', layout=field_layout, style=style)

# Buttons
save_button = widgets.Button(description='Speichern', style=style)
save_new_button = widgets.Button(description='Als neu speichern', style=style)
delete_button = widgets.Button(description='Löschen', style=style)

def on_save_clicked(b):
    add_thread_to_xml(xml_path, size.value, designation_de.value, designation_en.value, pitch.value,
                      major_dia_ext.value, pitch_dia_ext.value, minor_dia_ext.value,
                      major_dia_int.value, pitch_dia_int.value, minor_dia_int.value,
                      major_dia_4g6g.value, pitch_dia_4g6g.value, minor_dia_4g6g.value,
                      tap_drill.value)
    load_dropdown_options()
    thread_dropdown.value = designation_en.value  # ← Setze aktuelle Auswahl neu

def on_save_new_clicked(b):
    designation_en.value += "_neu"
    add_thread_to_xml(xml_path, size.value, designation_de.value, designation_en.value, pitch.value,
                      major_dia_ext.value, pitch_dia_ext.value, minor_dia_ext.value,
                      major_dia_int.value, pitch_dia_int.value, minor_dia_int.value,
                      major_dia_4g6g.value, pitch_dia_4g6g.value, minor_dia_4g6g.value,
                      tap_drill.value)
    load_dropdown_options()
    thread_dropdown.value = designation_en.value  # ← Setze neue Auswahl sicher
    
save_button.on_click(on_save_clicked)
save_new_button.on_click(on_save_new_clicked)
delete_button.on_click(on_delete_clicked)

# Button-Gruppe
button_box = VBox([save_button, save_new_button, delete_button])

# Neue Buttons
export_button = widgets.Button(description='Get XML from Fusion')
import_button = widgets.Button(description='Patch Fusion XML')
status_label = widgets.Label(value='⏳ Noch kein Export oder Import ausgeführt.')
venv_button = widgets.Button(description='Rebuild .venv')

# Letzter Importzeitpunkt
last_import_time = [None]

# Save-Buttons initial deaktivieren
save_button.disabled = True
save_new_button.disabled = True
delete_button.disabled = True

def on_venv_clicked(b):
    subprocess.run(['./bin/create_venv.sh'])
    print("✅ .venv wurde neu erstellt.")
    status_label.value = f"✅ .venv wurde neu erstellt am {datetime.datetime.now().strftime('%d.%m.%Y %H:%M:%S')}"
    
# Export-Callback
def on_export_clicked(b):
    subprocess.run(['./bin/sync_xml.sh', 'export'])
    save_button.disabled = False
    save_new_button.disabled = False
    delete_button.disabled = False
    load_dropdown_options()
    status_label.value = f"✅ Export aus Fusion abgeschlossen am {datetime.datetime.now().strftime('%d.%m.%Y %H:%M:%S')}"
    save_button.style.button_color = 'lightgreen'
    save_new_button.style.button_color = 'lightgreen'
    delete_button.style.button_color = 'lightgreen'
    thread_dropdown.add_class('highlighted')

# Import-Callback
def on_import_clicked(b):
    subprocess.run(['./bin/sync_xml.sh', 'import'])
    last_import_time[0] = datetime.datetime.now()
    update_status_label()
    status_label.value = f"✅ Import in Fusion abgeschlossen am {last_import_time[0].strftime('%d.%m.%Y %H:%M:%S')}"
    save_button.style.button_color = 'lightgreen'
    save_new_button.style.button_color = 'lightgreen'
    delete_button.style.button_color = 'lightgreen'
    # thread_dropdown.remove_class('highlighted')

# Zeitanzeige aktualisieren
def update_status_label():
    if last_import_time[0]:
        delta = datetime.datetime.now() - last_import_time[0]
        minutes = int(delta.total_seconds() // 60)
        status_label.value = f"In Fusion reimportiert am {last_import_time[0].strftime('%d.%m.%Y %H:%M:%S')} (vor {minutes} Minuten)"

# Periodisches Update starten
def periodic_update():
    while True:
        update_status_label()
        time.sleep(60)
        threading.Thread(target=periodic_update, daemon=True).start()

# Button-Callbacks verbinden
export_button.on_click(on_export_clicked)
import_button.on_click(on_import_clicked)
venv_button.on_click(on_venv_clicked)

# Control-Button-Reihe anzeigen
control_buttons = HBox([export_button, import_button, venv_button])
display(control_buttons)

# Statusanzeige anzeigen
display(status_label)

# XML-Formatierung
def pretty_print_xml(path):
    with open(path, 'r') as f:
        xml_string = f.read()
    parsed = xml.dom.minidom.parseString(xml_string)
    pretty_xml_as_string = parsed.toprettyxml(indent="  ")
    with open(path, 'w') as f:
        f.write(pretty_xml_as_string)

# Layout zusammenbauen (angepasst!)
top_row = HBox([
    VBox([thread_dropdown, size, designation_de, designation_en, pitch], layout=vbox_layout),
    button_box
])

# Anzeige
display(
    top_row,
    label_ext_6g, major_dia_ext, pitch_dia_ext, minor_dia_ext,
    label_int_6H, major_dia_int, pitch_dia_int, minor_dia_int, tap_drill,
    label_ext_4g6g, major_dia_4g6g, pitch_dia_4g6g, minor_dia_4g6g
)

# Sprachumschalter
current_language = {'lang': 'DE'}

labels = {
    'DE': {
        'thread_dropdown': 'Vorhandene Gewinde:',
        'size': 'Nenndurchmesser (Size):',
        'designation_de': 'Eigene Gewindebez. (Freitext):',
        'designation_en': 'Technische Gewindebez. (z.B. M56x0.75):',
        'pitch': 'Steigung (Pitch):',
        'ext_6g': '<b>External 6g</b>',
        'int_6H': '<b>Internal 6H</b>',
        'ext_4g6g': '<b>External 4g6g</b>',
        'save': 'Speichern',
        'save_new': 'Als neu speichern',
        'delete': 'Löschen',
        'export': 'Get XML from Fusion',
        'import': 'Patch Fusion XML',
        'venv': 'Rebuild .venv',
        'status': '⏳ Noch kein Export oder Import ausgeführt.'
    },
    'EN': {
        'thread_dropdown': 'Existing Threads:',
        'size': 'Nominal Diameter (Size):',
        'designation_de': 'Custom Thread designation (free text, CTD):',
        'designation_en': 'Technical Thread designation (e.g. M56x0.75):',
        'pitch': 'Pitch:',
        'ext_6g': '<b>External 6g</b>',
        'int_6H': '<b>Internal 6H</b>',
        'ext_4g6g': '<b>External 4g6g</b>',
        'save': 'Save',
        'save_new': 'Save as New',
        'delete': 'Delete',
        'export': 'Get XML from Fusion',
        'import': 'Patch Fusion XML',
        'venv': 'Rebuild .venv',
        'status': '⏳ No export or import done yet.'
    }
}

toggle_lang_button = widgets.Button(description='Switch to English')

def update_labels():
    lang = current_language['lang']
    thread_dropdown.description = labels[lang]['thread_dropdown']
    size.description = labels[lang]['size']
    designation_de.description = labels[lang]['designation_de']
    designation_en.description = labels[lang]['designation_en']
    pitch.description = labels[lang]['pitch']
    label_ext_6g.value = labels[lang]['ext_6g']
    label_int_6H.value = labels[lang]['int_6H']
    label_ext_4g6g.value = labels[lang]['ext_4g6g']
    save_button.description = labels[lang]['save']
    save_new_button.description = labels[lang]['save_new']
    delete_button.description = labels[lang]['delete']
    export_button.description = labels[lang]['export']
    import_button.description = labels[lang]['import']
    venv_button.description = labels[lang]['venv']
    status_label.value = labels[lang]['status']

def on_toggle_lang_clicked(b):
    if current_language['lang'] == 'DE':
        current_language['lang'] = 'EN'
        toggle_lang_button.description = 'Auf Deutsch umschalten'
    else:
        current_language['lang'] = 'DE'
        toggle_lang_button.description = 'Switch to English'
    update_labels()

toggle_lang_button.on_click(on_toggle_lang_clicked)

# Zeige Sprachumschalter an
display(toggle_lang_button)
update_labels()

load_dropdown_options()
thread_dropdown.observe(fill_fields_from_selection, names='value')
display(widgets.HTML(value="<div style='margin-left:20px; margin-bottom:20px;'></div>"))

HTML(value='<h1>Fusion 360 Gewindetool – XML-Editor mit Jupyter Notebook</h1>')

Label(value='⏳ Noch kein Export oder Import ausgeführt.')

HTML(value='<div style="text-align: right; width: 100%;"><b>External 6g</b></div>')

FloatText(value=0.0, description='MajorDia:', layout=Layout(max_width='500px', min_width='500px', width='500px…

FloatText(value=0.0, description='PitchDia:', layout=Layout(max_width='500px', min_width='500px', width='500px…

...

FloatText(value=0.0, description='MinorDia:', layout=Layout(max_width='500px', min_width='500px', width='500px…

Button(description='Switch to English', style=ButtonStyle())

HTML(value="<div style='margin-left:20px; margin-bottom:20px;'></div>")

TraitError: Invalid selection: value not found

TraitError: Invalid selection: value not found